In [55]:
import pandas as pd
import numpy as np
import random
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.environ import SolverFactory
import yfinance as yf
import matplotlib.pyplot as plt


In [56]:
model = pyo.ConcreteModel()

model.PLANTACAO = pyo.Set(initialize=['milho','trigo','soja'])

model.x = pyo.Var(model.PLANTACAO, bounds=(0,None), domain=NonNegativeIntegers)

#Dicionarios

custo_total ={
    'milho':2000,
    'trigo':2400,
    'soja':1400
}

homens_dia={
    'milho':20,
    'trigo':30,
    'soja':24
}

lucro = {
    'milho':600,
    'trigo':800,
    'soja':400
}


#objetivo
#maximizar lucro

def objetivo1(model):

    return sum(
        model.x[p] * lucro[p] for p in model.PLANTACAO
    )
model.objetivo = pyo.Objective(rule=objetivo1, sense=pyo.maximize)

#Restrições

def area_maxima(model):

    return sum(
        model.x[p] for p in model.PLANTACAO
        ) <= 400
model.area_maxima = pyo.Constraint(rule=area_maxima)

def custo_producao_total(model):

    return sum(model.x[p] * custo_total[p] for p in model.PLANTACAO) <= 800000
model.custo_producao = pyo.Constraint(rule=custo_producao_total)

def h_dia_total(model):

    return sum(model.x[p] * homens_dia[p] for p in model.PLANTACAO) <=7200
model.h_dia_total = pyo.Constraint(rule=h_dia_total)


In [57]:
model.pprint()

1 Set Declarations
    PLANTACAO : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'milho', 'trigo', 'soja'}

1 Var Declarations
    x : Size=3, Index=PLANTACAO
        Key   : Lower : Value : Upper : Fixed : Stale : Domain
        milho :     0 :  None :  None : False :  True : NonNegativeIntegers
         soja :     0 :  None :  None : False :  True : NonNegativeIntegers
        trigo :     0 :  None :  None : False :  True : NonNegativeIntegers

1 Objective Declarations
    objetivo : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 600*x[milho] + 800*x[trigo] + 400*x[soja]

3 Constraint Declarations
    area_maxima : Size=1, Index=None, Active=True
        Key  : Lower : Body                          : Upper : Active
        None :  -Inf : x[milho] + x[trigo] + x[soja] : 400.0 :   True
    custo_producao : Size=1, Index=None, Active=True
     

In [58]:
opt = SolverFactory('cplex', executable='C:\\CPLEX_Studio2211\\cplex\\bin\\x64_win64\\cplex.exe')
opt.options['TimeLimit'] = 180 
results = opt.solve(model, tee=True, warmstart=True)
print("-------FIM----------")



Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\joaon\AppData\Local\Temp\tmpbplm9de4.cplex.log' open.
CPLEX> New value for time limit in seconds: 180
CPLEX> Problem 'C:\Users\joaon\AppData\Local\Temp\tmp48mz3qcq.pyomo.lp' read.
Read time = 0.00 sec. (0.00 ticks)
CPLEX> Warning: no MIP start values read, no MIP start loaded.
MIP start file 'C:\Users\joaon\AppData\Local\Temp\tmpsbw438j6.cplex.mst' read.
CPLEX> Problem name         : C:\Users\joaon\AppData\Local\Temp\tmp48mz3qcq.pyomo.lp
Objective sense      : Maximize
Variables            :       3  [General Integer: 3]
Objective nonzeros   :       3
Linear constraints   :       3  [

In [59]:
pyo.value(model.objetivo)

216000.0

In [60]:
for m in model.PLANTACAO:
    print(f"Quantidade HA de: {m}: ", pyo.value(model.x[m]))

Quantidade HA de: milho:  360.0
Quantidade HA de: trigo:  0.0
Quantidade HA de: soja:  0.0
